<a href="https://colab.research.google.com/github/Sharon-Tseng/VincentMobile_Database_System/blob/main/Customer_and_Member_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer and Member

In [2]:
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
! ldconfig

# -- Include cx_Oracle package --
! pip install cx_Oracle

--2025-04-24 13:50:54--  https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
Resolving download.oracle.com (download.oracle.com)... 23.46.192.121
Connecting to download.oracle.com (download.oracle.com)|23.46.192.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75462547 (72M) [application/zip]
Saving to: ‘oracleinstantclient.zip’

oracleinstantclient 100%[===================>]  71.97M  78.5MB/s    in 0.9s    

2025-04-24 13:50:56 (78.5 MB/s) - ‘oracleinstantclient.zip’ saved [75462547/75462547]

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/l

In [3]:
# Import cx_Oracle package and connect to Oracle Database
import cx_Oracle

HOST_NAME = "imz409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "LA302"
PASSWORD = "CJ995"

dsn_tns = cx_Oracle.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME)
conn = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns)

c = conn.cursor()   # c: represent database connection

In [4]:
# Import required packages
import cx_Oracle
from google.colab.widgets import TabBar
import ipywidgets as widgets
import time
import pandas as pd
from ipywidgets import interact
from ipywidgets import Text
from ipywidgets import DatePicker
from ipywidgets import IntText
from ipywidgets import Label
from datetime import date, datetime
import re
from logging import disable
from pickle import FALSE

In [8]:
# Tab object
tb_all = TabBar(['Introduction','Register',
          'Login','Welcome','Forget_Password',
          'Change_Password','Update_Personal_Account',
          'Search_Product','Show_Product_Detail','Shopping Cart','Checkout','Order List', 'Order Detail', 'Trade_In_Detail'])

# Define global variables for member information and order tracking
member_id = widgets.Label("")
selected_order_id = widgets.Label("")
order_id = widgets.Label("")

def showPage(tabName):
  with tb_all.output_to(tabName):
    if tabName == "Search_Product":
      search_product_textbox.value = ''
      sort_by_category_dropdown.value = None
      sort_by_price_dropdown.value = None
      initialize_records()
      on_navigate_to_search_product()
    elif tabName == "Shopping Cart":
      show_cart_items()
    elif tabName == "Checkout":
      prepare_checkout()
    elif tabName == "Order List":
      show_orders()
    elif tabName == "Order Detail":
      show_order_detail()
    elif tabName == "Trade_In_Detail":
      show_trade_in_detail()
    display()

# Input validations:
def is_valid_username(username):
  return re.match(r"^[a-zA-Z0-9]{3,30}$",username)

def is_valid_name(name): # firstname_textbox.value, lastname_textbox.value
  return re.match(r"^[a-zA-Z0-9]{1,50}$",name)

def is_valid_pwd(pwd):
  return 6 <= len(pwd) <= 30

def is_valid_address(address):
  return 1 <= len(address) <= 100

def is_valid_answer(security_answer):
  return re.match(r"^[a-zA-Z0-9]{1,50}$",security_answer)

def is_valid_creditno(credit_no):
  return re.match(r"^[0-9]{8,20}$",credit_no)

def is_valid_cvc(cvc):
  return re.match(r"^[0-9]{3,4}$",cvc)

def is_valid_phone(phone):
  return re.match(r"^[0-9]{8,8}$",phone)

def is_valid_email(email):
  return re.match(r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$", email)

##############################################
############## Introduction Tab ###########
##############################################

with tb_all.output_to('Introduction'):
  intro_label = Label("Welcome to 302 Vincent Mobile")

  btn_RegisterPage = widgets.Button(description="Register as Member")
  btn_LoginPage = widgets.Button(description="Login")

  def on_btnRegisterPage_button_clicked(b):
    showPage("Register")
  btn_RegisterPage.on_click(on_btnRegisterPage_button_clicked)

  def on_btn_LoginPage_button_clicked(b):
    showPage("Login")
  btn_LoginPage.on_click(on_btn_LoginPage_button_clicked)

  intro_row1HBox = widgets.HBox([intro_label])
  intro_row2HBox = widgets.HBox([btn_LoginPage])
  intro_row3HBox = widgets.HBox([btn_RegisterPage])

  display(widgets.VBox([intro_row1HBox, intro_row2HBox, intro_row3HBox]))

##############################################
############## Register Tab ###############
##############################################

with tb_all.output_to("Register"):
  reg_label = Label("Please fill in the information")
  username_label = Label("Username: ")
  username_textbox = Text(placeholder='Enter your username', disabled=False)
  password_label = Label("Password: ")
  password_textbox = Text(placeholder='Enter your password', disabled=False)
  firstname_label = Label("First Name: ")
  firstname_textbox = Text(placeholder='Tai Ming', disabled=False)
  lastname_label = Label("Last Name: ")
  lastname_textbox = Text(placeholder='Wong', disabled=False)
  address_label = Label("Address: ")
  address_textbox = Text(placeholder='Sai Kung, Hong Kong', disabled=False)
  gender_label = Label("Gender: ")
  gender_dropdown = widgets.Dropdown(options=["Male", "Female", "Others"], value="Male")
  dob_label = Label("Date of Birth: ")
  dob_datepicker = DatePicker(placeholder='2000/01/01', disabled=False, value= date.today())
  phone_label = Label("Phone: ")
  phone_textbox = Text(placeholder='12345678', disabled=False)
  email_label = Label("Email: ")
  email_textbox = Text(placeholder='e.g, example@domain.com', disabled=False)
  creditno_label = Label("Credit No.: ")
  creditno_textbox = Text(placeholder='1234567891234567', disabled=False)
  security_question_label = Label("Security Question: ")
  cmb = widgets.Dropdown(options=["What is your favorite animal?", "Which city did you live when you were 10?",
                    "What year is your grandmother born?", "What is the name of your first pet?",
                    "What is your favorite sport?"], value = "What is your favorite animal?")
  security_answer_label = Label("Security Answer: ")
  security_answer_textbox = Text(placeholder='Security Answer', disabled=False)
  expiry_date_label = Label("Expiry Date: ")
  expiry_date_datepicker = DatePicker(placeholder='1991-01-01', disabled=False, value= date.today())
  reg_cvc_label = Label("CVC: ")
  reg_cvc_textbox = Text(placeholder='000', disabled=False)

  reg_status = Label("")
  reg_register_button = widgets.Button(description="Register")
  reg_back_button = widgets.Button(description="Back")


  # Register Button - event handler
  def on_register_button_clicked(b):

    all_input_values = [username_textbox.value, password_textbox.value, firstname_textbox.value, lastname_textbox.value, address_textbox.value,
             phone_textbox.value, email_textbox.value, creditno_textbox.value, reg_cvc_textbox.value, security_answer_textbox.value]


    for i in all_input_values:
      if i == "":
        reg_status.value = "❗Missing Value. Please input all items"
        return

    try:
      if not is_valid_username(username_textbox.value):
        reg_status.value = "❗Username must be 3-30 characters, letters and numbers only."
        return

      if not is_valid_pwd(password_textbox.value):
        reg_status.value = "❗Password must be 6-30 characters, letters and numbers only."
        return

      if not is_valid_name(firstname_textbox.value):
        reg_status.value = "❗Firstname must be 1-50 characters only."
        return

      if not is_valid_name(lastname_textbox.value):
        reg_status.value = "❗Lastname must be 1-50 characters only."
        return


      if not is_valid_address(address_textbox.value):
        reg_status.value = "❗Address must be within 100 characters."
        return

      if not is_valid_phone(phone_textbox.value):
        reg_status.value = "❗Phone must be 8 numbers only."
        return

      if not is_valid_email(email_textbox.value):
        reg_status.value = "❗Invalid email format. Please use example@domain.com"
        return

      if not is_valid_creditno(creditno_textbox.value):
        reg_status.value = "❗Credit card number must be 8-20 numbers only."
        return

      if not is_valid_cvc(reg_cvc_textbox.value):
        reg_status.value = "❗CVC must be 3-4 numbers only."
        return

      if not is_valid_answer(security_answer_textbox.value):
        reg_status.value = "❗Security answer must be 1-50 characters and numbers only."
        return


        # exclude duplicated username
      c.execute("SELECT M_USERNAME FROM MEMBER WHERE M_USERNAME = '{}'".format(username_textbox.value))
      existing_user = c.fetchone()

      if existing_user:
        reg_status.value = "Username already exists. Please create another username."
        return

      else:
        c.execute(
        """
        INSERT INTO MEMBER VALUES(seq_member.nextval, '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}' ,TO_DATE('{}', 'YYYY-MM-DD'), '{}', '{}', '{}', TO_DATE('{}', 'YYYY-MM-DD'), '{}')
        """.format(firstname_textbox.value, lastname_textbox.value,
              username_textbox.value, password_textbox.value, address_textbox.value,
              cmb.value, security_answer_textbox.value, creditno_textbox.value,
              expiry_date_datepicker.value, reg_cvc_textbox.value, phone_textbox.value, email_textbox.value,
              dob_datepicker.value, gender_dropdown.value)
          )

        c.execute("commit")

        username_textbox.value = ""
        password_textbox.value = ""
        firstname_textbox.value = ""
        lastname_textbox.value = ""
        address_textbox.value = ""
        phone_textbox.value = ""
        email_textbox.value = ""
        creditno_textbox.value = ""
        security_answer_textbox.value=""
        reg_cvc_textbox.value=""


        reg_register_button.disabled=True
        reg_status.value = "Successfully Registered!"

    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      reg_status.value = f"Database Error:{error_obj.message}"

  reg_register_button.on_click(on_register_button_clicked)

  def on_back_button_clicked(b):
    showPage("Introduction")
  reg_back_button.on_click(on_back_button_clicked)

  # display objects
  reg_msg_labelVBox = widgets.VBox([reg_label])
  reg_labelVBox = widgets.VBox([username_label, password_label, firstname_label, lastname_label,
                  address_label,gender_label,dob_label,phone_label,email_label,
                  creditno_label,reg_cvc_label,security_question_label,security_answer_label,expiry_date_label])
  reg_textboxVBox = widgets.VBox([username_textbox, password_textbox, firstname_textbox, lastname_textbox,
                  address_textbox,gender_dropdown,dob_datepicker,phone_textbox,email_textbox,
                  creditno_textbox,reg_cvc_textbox,cmb,security_answer_textbox,expiry_date_datepicker])
  reg_line1HBox = widgets.HBox([reg_msg_labelVBox])
  reg_line2HBox = widgets.HBox([reg_labelVBox, reg_textboxVBox])
  reg_line3HBox = widgets.HBox([reg_register_button, reg_back_button])
  reg_line4HBox = widgets.HBox([reg_status])

  display(widgets.VBox([reg_line1HBox,reg_line2HBox, reg_line3HBox,reg_line4HBox]))

##############################################
############## Login Tab ##################
##############################################

with tb_all.output_to('Login'):
  login_label = Label("Please login with your username and password:")
  login_un_label = Label("Username: ")
  login_un_textbox = Text(placeholder='Enter your username', disabled=False)
  login_pwd_label = Label("Password: ")
  login_pwd_textbox = Text(placeholder='Enter your password', disabled=False)

  login_status = Label("")
  login_button = widgets.Button(description="Login")
  forget_pwd_button = widgets.Button(description="Forget Password")
  login_back_button = widgets.Button(description="Back")

  member_username = widgets.Label("")
  member_pwd = widgets.Label("")

  def on_button_login_clicked(b):
    c.execute(
    """
    SELECT M_ID
    FROM MEMBER
    WHERE M_USERNAME = :username AND M_PW = :password
    """,{'username': login_un_textbox.value, 'password': login_pwd_textbox.value}
    )

    member_username.value = login_un_textbox.value
    member_pwd.value = login_pwd_textbox.value

    login_un_textbox.value = ""
    login_pwd_textbox.value = ""

    row = c.fetchone()

    if row:
      member_id.value = str(row[0])
      showPage("Welcome")
    else:
      login_status.value = "❗Incorrect username or password! Please try again."
      showPage("Login")


  login_button.on_click(on_button_login_clicked)

  def on_back_button_clicked(b):
    showPage("Introduction")
  login_back_button.on_click(on_back_button_clicked)

  # design login interface
  login_msg_labelVBox = widgets.VBox([login_label])
  login_labelVBox = widgets.VBox([login_un_label,login_pwd_label])
  login_textboxVBox = widgets.VBox([login_un_textbox,login_pwd_textbox])

  login_line1HBox = widgets.HBox([login_msg_labelVBox])
  login_line2HBox = widgets.HBox([login_labelVBox, login_textboxVBox])
  login_line3HBox = widgets.HBox([login_button, forget_pwd_button, login_back_button])
  login_line4HBox = widgets.HBox([login_status])

  display(widgets.VBox([login_line1HBox, login_line2HBox, login_line3HBox, login_line4HBox]))

  def on_forget_pwd_clicked(b):
    showPage("Forget_Password")
  forget_pwd_button.on_click(on_forget_pwd_clicked)

##############################################
########## Forget_Password Tab ############
##############################################

with tb_all.output_to('Forget_Password'):
  fpwd_label = Label("Please fill in the below information to verify your identity:")
  fpwd_un_textbox = Text(value='', placeholder='Enter your username', description='Username:', disabled=False)
  spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")
  fpwd_question_label = Label("Security Question")
  fpwd_question_textbox = Text(placeholder='e.g, What is your favorite animal?', disabled = True)
  fpwd_answer_label = Label("Security Question Answer")
  fpwd_ans_textbox = Text(value='',placeholder='e.g, dog', disabled=False)

  fpwd_status = Label("")
  member_username_2 = widgets.Label("")

  retrieve_button = widgets.Button(description="Retrieve")
  back_button = widgets.Button(description="Back")
  confirm_button = widgets.Button(description="Confirm")

  # retrieve button
  def on_retrieve_button_clicked(b):
    if fpwd_un_textbox.value == "":
      fpwd_status.value = "❗Please enter a valid username!"
    else:
      c.execute("SELECT M_QUESTION FROM MEMBER WHERE M_USERNAME = '{}'".format(fpwd_un_textbox.value))

      row = c.fetchone()

      if row:
        if row[0] is not None:
          fpwd_question_textbox.value = row[0]
          on_retrieve_button_clicked.disabled = True
          member_username_2.value = fpwd_un_textbox.value
          fpwd_status.value = ""
      else:
        fpwd_status.value = "❗Username not found"
  retrieve_button.on_click(on_retrieve_button_clicked)

  def on_confirm_button_clicked(b):
    c.execute(
    """
    SELECT M_ANSWER FROM MEMBER
    WHERE M_USERNAME = '{}' AND M_QUESTION = '{}'
    """.format(fpwd_un_textbox.value,fpwd_question_textbox.value))

    for row in c:
      if fpwd_ans_textbox.value == row[0]:
        showPage('Change_Password')
      else:
        fpwd_status.value = "❗Incorrect answer! Please try again."

    fpwd_un_textbox.value = ""
    fpwd_question_textbox.value = ""
    fpwd_ans_textbox.value= ""

  confirm_button.on_click(on_confirm_button_clicked)


  # back button
  def on_back_button_clicked(b):
    showPage("Login")
  back_button.on_click(on_back_button_clicked)

  line1HBox = widgets.HBox([fpwd_label])
  line2HBox = widgets.HBox([fpwd_un_textbox, retrieve_button])
  line3HBox = widgets.HBox([fpwd_status])
  line4HBox = widgets.HBox([spacer])
  line5HBox = widgets.HBox([fpwd_question_label, fpwd_question_textbox])
  line6HBox = widgets.HBox([fpwd_answer_label, fpwd_ans_textbox])
  line7HBox = widgets.HBox([back_button, confirm_button])

  display(widgets.VBox([line1HBox,line2HBox,line4HBox,line5HBox,line6HBox,line7HBox,line3HBox]))

##############################################
########## Change PWD Tab ################
##############################################

with tb_all.output_to('Change_Password'):
  cpwd_label = Label("Please fill in the below information to reset your password:")
  new_pwd_label = Label("New Password:")
  new_pwd_textbox = Text(placeholder='Your new password', disabled=False)
  retype_label = Label("Retype:")
  retype_textbox = Text(placeholder='Retype your new password', disabled=False)

  cpwd_msg_label = Label("")
  update_button = widgets.Button(description='Update')
  back_button = widgets.Button(description='Back')

  def on_update_button_clicked(b):
    all_input_values = [new_pwd_textbox.value, retype_textbox.value]
    # check null values

    for i in all_input_values:
      if i == "":
        cpwd_msg_label.value = "❗Missing Value. Please input all items"
        return
      if not is_valid_pwd(i):
        cpwd_msg_label.value = "❗Password must be 6-30 characters only."
        return

    if new_pwd_textbox.value != retype_textbox.value:
      cpwd_msg_label.value = "❗Retype password is not the same as your new password! Please check again!"
      return

    # check whether new password is the same as the original one
    c.execute(
        """
        SELECT M_PW FROM MEMBER
        WHERE M_USERNAME = '{}'
        """.format(member_username_2.value)
    )
    original_pw = c.fetchone()[0]
    if original_pw == new_pwd_textbox.value:
      cpwd_msg_label.value = "❗New password cannot be the same as the current one!"
      return

    c.execute(
    """
    UPDATE MEMBER
    SET M_PW = '{}'
    WHERE M_USERNAME = '{}'
    """.format(new_pwd_textbox.value, member_username_2.value)
    )
    c.execute("commit")
    cpwd_msg_label.value = "✅ Successfully Updated!"
  update_button.on_click(on_update_button_clicked)

  def on_back_button_clicked(b):
    showPage("Login")
  back_button.on_click(on_back_button_clicked)

  line1HBox = widgets.HBox([cpwd_label])
  line2HBox = widgets.HBox([new_pwd_label,new_pwd_textbox])
  line3HBox = widgets.HBox([retype_label,retype_textbox])
  line4HBox = widgets.HBox([update_button, back_button])
  line5HBox = widgets.HBox([cpwd_msg_label])

  display(widgets.VBox([line1HBox,line2HBox,line3HBox,line4HBox,line5HBox]))

##############################################
############## Welcome Tab ################
##############################################

with tb_all.output_to('Welcome'):
  welcome_label = Label("Welcome! Start browsing with the following functions:")
  change_info_btn = widgets.Button(description='Manage My Account')
  search_prod_btn = widgets.Button(description='Search Products')
  shopping_cart_btn = widgets.Button(description='Shopping Cart')
  history_btn = widgets.Button(description='Purchase History')

  def on_change_info_btn_click(b):
    showPage('Update_Personal_Account')
  change_info_btn.on_click(on_change_info_btn_click)

  def on_search_produc_btn_click(b):
    showPage('Search_Product')
  search_prod_btn.on_click(on_search_produc_btn_click)

  def on_shopping_cart_btn_click(b):
    showPage('Shopping Cart')
  shopping_cart_btn.on_click(on_shopping_cart_btn_click)

  def on_history_btn_click(b):
    showPage('Order List')
  history_btn.on_click(on_history_btn_click)

  display(widgets.VBox([welcome_label, change_info_btn, search_prod_btn, shopping_cart_btn, history_btn]))

##############################################
#### Update_Personal_Account Tab ##########
##############################################

with tb_all.output_to('Update_Personal_Account'):
  m_id_textbox = Text(description='Member_ID', disabled=True)
  up_username_label = Label("Username: ")
  up_username_textbox = Text(placeholder='Your Username', disabled=True)
  up_password_label = Label("Password: ")
  up_password_textbox = Text(placeholder='Your Password', disabled=True)
  up_firstname_label = Label("First Name: ")
  up_firstname_textbox = Text(placeholder='Tai Ming', disabled=True)
  up_lastname_label = Label("Last Name: ")
  up_lastname_textbox = Text(placeholder='Wong', disabled=True)
  up_address_label = Label("Address: ")
  up_address_textbox = Text(placeholder='Sai Kung, Hong Kong', disabled=True)
  up_gender_label = Label("Gender: ")
  up_gender_dropdown = widgets.Dropdown(options=["Male", "Female", "Others"],disabled=True)
  up_dob_label = Label("Date of Birth: ")
  up_dob_datepicker = DatePicker(placeholder='2000/01/01', disabled=True)
  up_phone_label = Label("Phone: ")
  up_phone_textbox = Text(placeholder='12345678', disabled=True)
  up_email_label = Label("Email: ")
  up_email_textbox = Text(placeholder='e.g, example@domain.com', disabled=True)
  up_creditno_label = Label("Credit No.: ")
  up_creditno_textbox = Text(placeholder='1234567891234567', disabled=True)
  up_security_question_label = Label("Security Question: ")
  up_cmb = widgets.Dropdown(options=["What is your favorite animal?", "Which city did you live when you were 10?",
                    "What year is your grandmother born?", "What is the name of your first pet?",
                    "What is your favorite sport?"],disabled=True)
  up_security_answer_label = Label("Security Answer: ")
  up_security_answer_textbox = Text(placeholder='Security Answer', disabled=True)
  up_expiry_date_label = Label("Expiry Date: ")
  up_expiry_date_datepicker = DatePicker(placeholder='1991-01-01', disabled=True)
  up_cvc_label = Label("CVC: ")
  up_cvc_textbox = Text(placeholder='000', disabled=True)

  up_status = Label("")
  retrieve_btn = widgets.Button(description = "Retrieve Info.")
  spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")
  update_btn = widgets.Button(description = "Update")
  up_back_btn = widgets.Button(description = "Back")

  # define retrieve info
  def on_retrieve_button_clicked(b):
    up_password_textbox.disabled = False
    up_firstname_textbox.disabled = False
    up_lastname_textbox.disabled = False
    up_address_textbox.disabled = False
    up_phone_textbox.disabled = False
    up_email_textbox.disabled = False
    up_creditno_textbox.disabled = False
    up_security_answer_textbox.disabled = False
    up_cvc_textbox.disabled = False
    up_gender_dropdown.disabled = False
    up_cmb.disabled = False
    up_expiry_date_datepicker.disabled = False
    up_gender_dropdown.disabled = False
    up_dob_datepicker.disabled = False

    if up_username_textbox == "":
      up_status.value = "❗Missing Username. Please input all items"
      return
    else:
      c.execute("SELECT * FROM MEMBER WHERE M_USERNAME= '{}' AND M_PW = '{}'".format(member_username.value, member_pwd.value))
      for row in c:
        m_id_textbox.value = str(row[0])
        up_username_textbox.value = row[3]
        up_password_textbox.value = row[4]
        up_firstname_textbox.value = row[1]
        up_lastname_textbox.value = row[2]
        up_address_textbox.value = row[5]
        up_gender_dropdown.value = row[14]
        up_dob_datepicker.value = row[13].date()
        up_phone_textbox.value = row[11]
        up_email_textbox.value = row[12]
        up_creditno_textbox.value = row[8]
        up_cmb.value = row[6]
        up_security_answer_textbox.value = row[7]
        up_expiry_date_datepicker.value =row[9].date()
        up_cvc_textbox.value = row[10]

  retrieve_btn.on_click(on_retrieve_button_clicked)

  # define update info
  def on_update_btn_clicked(b):

    all_input_values = [up_password_textbox.value, up_firstname_textbox.value, up_lastname_textbox.value, up_address_textbox.value,
             up_phone_textbox.value, up_email_textbox.value, up_creditno_textbox.value, up_cvc_textbox.value, up_security_answer_textbox.value]

    # check null values
    for i in all_input_values:
      if i == "":
        up_status.value = "❗Missing Value. Please input all items"
        return

    try:
      if not is_valid_pwd(up_password_textbox.value):
        up_status.value = "❗Password must be 6-30 characters only."
        return

      if not is_valid_name(up_firstname_textbox.value):
        up_status.value = "❗Firstname must be 1-50 characters only."
        return

      if not is_valid_name(up_lastname_textbox.value):
        up_status.value = "❗Lastname must be 1-50 characters only."
        return


      if not is_valid_address(up_address_textbox.value):
        up_status.value = "❗Address must be within 100 characters."
        return

      if not is_valid_phone(up_phone_textbox.value):
        up_status.value = "❗Phone must be 8 numbers only."
        return

      if not is_valid_email(up_email_textbox.value):
        up_status.value = "❗Invalid email format. Please use example@domain.com"
        return

      if not is_valid_creditno(up_creditno_textbox.value):
        up_status.value = "❗Credit card number must be 8-20 numbers only."
        return

      if not is_valid_cvc(up_cvc_textbox.value):
        up_status.value = "❗CVC must be 3-4 numbers only."
        return

      if not is_valid_answer(up_security_answer_textbox.value):
        up_status.value = "❗Security answer must be 1-50 characters and numbers only."
        return


      c.execute(
      """
      Update MEMBER
      SET M_USERNAME = '{}',
      M_PW = '{}',
      M_FIRST_NAME = '{}',
      M_LAST_NAME = '{}',
      M_ADDRESS = '{}',
      M_GENDER = '{}',
      M_DOB = TO_DATE('{}', 'YYYY-MM-DD'),
      M_PHONE = '{}',
      M_EMAIL = '{}',
      M_CREDIT_NO = '{}',
      M_QUESTION = '{}',
      M_ANSWER = '{}',
      M_CREDIT_EXPIRE = TO_DATE('{}', 'YYYY-MM-DD'),
      M_CREDIT_CVC = '{}'
      WHERE M_ID = {}
      """.format(up_username_textbox.value,up_password_textbox.value,up_firstname_textbox.value,up_lastname_textbox.value,up_address_textbox.value,up_gender_dropdown.value,
            up_dob_datepicker.value,up_phone_textbox.value,up_email_textbox.value,up_creditno_textbox.value,up_cmb.value,
            up_security_answer_textbox.value,up_expiry_date_datepicker.value,up_cvc_textbox.value,m_id_textbox.value)
      )
      c.execute("commit")

      up_status.value = "✅ Successfully Updated!"

      m_id_textbox.value = ""
      up_username_textbox.value = ""
      up_password_textbox.value = ""
      up_firstname_textbox.value = ""
      up_lastname_textbox.value = ""
      up_address_textbox.value = ""
      up_phone_textbox.value = ""
      up_email_textbox.value = ""
      up_creditno_textbox.value = ""
      up_security_answer_textbox.value=""
      up_cvc_textbox.value=""
      up_gender_dropdown.value = None
      up_cmb.value = None
      up_dob_datepicker.value = None
      up_expiry_date_datepicker.value = None

    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      up_status.value = f"Database Error:{error_obj.message}"

  update_btn.on_click(on_update_btn_clicked)

  def on_up_back_btn_clicked(b):
    showPage("Welcome")
    m_id_textbox.value = ""
    up_username_textbox.value = ""
    up_password_textbox.value = ""
    up_firstname_textbox.value = ""
    up_lastname_textbox.value = ""
    up_address_textbox.value = ""
    up_phone_textbox.value = ""
    up_email_textbox.value = ""
    up_creditno_textbox.value = ""
    up_security_answer_textbox.value=""
    up_cvc_textbox.value=""
    up_gender_dropdown.value = None
    up_cmb.value = None
    up_dob_datepicker.value = None
    up_expiry_date_datepicker.value = None
  up_back_btn.on_click(on_up_back_btn_clicked)

  line1HBox = widgets.HBox([m_id_textbox, retrieve_btn])
  line_label_VBox = widgets.VBox([up_username_label, up_password_label, up_firstname_label, up_lastname_label,
                  up_address_label,up_gender_label,up_dob_label,up_phone_label,up_email_label,
                  up_creditno_label,up_cvc_label,up_security_question_label,up_security_answer_label,up_expiry_date_label])
  line_textbox_VBox = widgets.VBox([up_username_textbox, up_password_textbox, up_firstname_textbox, up_lastname_textbox,
                  up_address_textbox,up_gender_dropdown,up_dob_datepicker,up_phone_textbox,up_email_textbox,
                  up_creditno_textbox,up_cvc_textbox,up_cmb,up_security_answer_textbox,up_expiry_date_datepicker])
  line2_HBox = widgets.HBox([line_label_VBox,line_textbox_VBox])
  line3_HBox = widgets.HBox([up_back_btn, update_btn])
  line4HBox = widgets.HBox([up_status])

  display(widgets.VBox([line1HBox,spacer,line2_HBox,line3_HBox,line4HBox]))

##############################################
############ Search_Product Tab ###########
##############################################

with tb_all.output_to("Search_Product"):

  search_box = widgets.VBox(
      layout = widgets.Layout(
          border = '2px solid gray',
          padding = '10px',
          margin = '10px',
          width = 'auto'
      )
  )
  # Allow member to search products by keyword
  search_product_title = widgets.HTML(value="<strong> Search products by entering product ID or name. Sort by category or price.</strong>", layout={'margin': '5px 0'})
  search_product_textbox = Text(placeholder="Enter product ID, or name", layout={'width': '200px', 'margin': '5px'}, disabled = False)
  search_product_label = Label("Search by:")
  search_status_label = Label("")
  search_button = widgets.Button(description="Search all",button_style='info', layout={'width': '150px', 'margin': '5px'})

  keyword_search_box = widgets.VBox([search_product_title,
                     widgets.HBox([search_product_label,search_product_textbox, search_button])])

  # Create dynamic combo box for search by category
  sort_cat_title = Label("Search by category:")
  cmb_options = []
  cmb_options.insert(0, "All categories")

  c.execute("SELECT DISTINCT(P_CATEGORY) FROM PRODUCT")
  for row in c:
    cmb_options.append(row[0])
  sort_by_category_dropdown = widgets.Dropdown(options = cmb_options, layout={'width': '150px', 'margin': '5px'}, value = None)
  sortbycategory_button = widgets.Button(description="Search by category",button_style='info', layout={'width': '150px', 'margin': '5px'})

  category_search_box = widgets.HBox([sort_cat_title, sort_by_category_dropdown, sortbycategory_button])

  # Search by price
  sort_price_title = Label("Search by price:")
  cmb2_options = []
  interval_size = 5000

  c.execute("SELECT DISTINCT(P_PRICE) FROM PRODUCT")
  prices = [row[0] for row in c.fetchall()]

  max_price = max(prices)
  min_price = min(prices)

  for lower in range(int(min_price), int(max_price) + interval_size, interval_size):
    upper = lower + interval_size
    interval_label = f"{lower}~{upper-1}"
    cmb2_options.append((interval_label, (lower, upper-1)))

  cmb2_options.insert(0, ("All prices", (min_price, max_price)))

  sort_by_price_dropdown = widgets.Dropdown(options=cmb2_options,layout={'width': '180px', 'margin': '5px'}, value = None)
  sortbyprice_button = widgets.Button(description="Search by price",button_style='info', layout={'width': '150px', 'margin': '5px'})

  price_search_box = widgets.HBox([sort_price_title, sort_by_price_dropdown, sortbyprice_button])


  previous_5_button = widgets.Button(description="Previous 5 Items", button_style='info', layout={'width': '150px', 'margin': '5px'})
  next_5_button = widgets.Button(description="Next 5 Items", button_style='info', layout={'width': '150px', 'margin': '5px'})
  view_product_back_button = widgets.Button(description="Back")

  ret_5_id_textboxes = [Text(value='',description='Product ID:',placeholder='Not available',disabled=True) for i in range(5)]
  ret_5_name_textboxes = [Text(value='',description='Name:',placeholder='Not available',disabled=True) for i in range(5)]
  ret_5_category_textboxes = [Text(value='',description='Category:',placeholder='Not available',disabled=True) for i in range(5)]
  ret_5_price_textboxes = [Text(value='',description='Price:',placeholder='Not available',disabled=True) for i in range(5)]
  ret_5_detail_buttons = [widgets.Button(description='Product Details',disabled=True) for i in range(5)]
  ret_5_data_HBoxes = [widgets.HBox([ret_5_id_textboxes[i], ret_5_name_textboxes[i],ret_5_category_textboxes[i],
                    ret_5_price_textboxes[i], ret_5_detail_buttons[i]]) for i in range(5)]

  current_record_idx = widgets.Label("0")
  search_results = []
  all_records = []

  start_idx = 0
  total_records = 0
  current_search_type = None

  def initialize_records():
    global all_records, total_records, search_results
    c.execute("SELECT P_ID, P_NAME, P_CATEGORY,P_PRICE FROM PRODUCT ORDER BY P_ID")
    all_records = c.fetchall()
    total_records = len(all_records)
    search_results = None
    show_records()

  def clear_product_textbox():
    for i in range(5):
      ret_5_id_textboxes[i].value = ''
      ret_5_name_textboxes[i].value = ''
      ret_5_price_textboxes[i].value = ''
      ret_5_category_textboxes[i].value = ''
      ret_5_detail_buttons[i].disabled = True


  def update_button_state():
    previous_5_button.disabled = start_idx == 0
    next_5_button.disabled = (start_idx + 5 >= total_records)

  def show_records():
    global total_records

    records_to_show = search_results if search_results is not None else all_records
    total_records = len(records_to_show)

    clear_product_textbox()

    if total_records == 0 :
      search_status_label.value = "❗ No items found"
      next_5_button.disabled = True
      previous_5_button.disabled = True

      for i in range(5):
        ret_5_id_textboxes[i].layout.display = 'none'
        ret_5_name_textboxes[i].layout.display = 'none'
        ret_5_category_textboxes[i].layout.display = 'none'
        ret_5_price_textboxes[i].layout.display = 'none'
        ret_5_detail_buttons[i].layout.display = 'none'
      return

    else:
      for i in range(5):
          result_idx = start_idx + i
          if result_idx < total_records:
            row = records_to_show[result_idx]
            ret_5_id_textboxes[i].value = str(row[0])
            ret_5_name_textboxes[i].value = str(row[1])
            ret_5_price_textboxes[i].value = str(row[3] if len(row) > 3 else row[3])
            ret_5_category_textboxes[i].value = str(row[2] if len(row) > 3 else row[2])
            ret_5_detail_buttons[i].disabled = False

            ret_5_id_textboxes[i].layout.display = ''
            ret_5_name_textboxes[i].layout.display = ''
            ret_5_price_textboxes[i].layout.display = ''
            ret_5_category_textboxes[i].layout.display = ''
            ret_5_detail_buttons[i].layout.display = ''
          else:
            ret_5_id_textboxes[i].layout.display = 'none'
            ret_5_name_textboxes[i].layout.display = 'none'
            ret_5_price_textboxes[i].layout.display = 'none'
            ret_5_category_textboxes[i].layout.display = 'none'
            ret_5_detail_buttons[i].layout.display = 'none'

    update_button_state()
    end_idx = min(start_idx + 5, total_records)
    search_status_label.value = f"Showing {start_idx + 1}-{end_idx} of {total_records} products"

  def on_navigate_to_search_product():
    global search_results, all_records, start_idx, current_search_type

    search_results = None
    current_search_type = None
    start_idx = 0

    search_product_textbox.value = ''
    sort_by_category_dropdown.value = None
    sort_by_price_dropdown.value = None

    initialize_records()


  def on_next_button_clicked(b):
    global start_idx
    if start_idx + 5 < total_records:
      start_idx += 5
      show_records()
  next_5_button.on_click(on_next_button_clicked)

  def on_prev_button_clicked(b):
    global start_idx
    if start_idx - 5 >= 0:
      start_idx -= 5
    else:
      start_idx = 0
    show_records()
  previous_5_button.on_click(on_prev_button_clicked)

  def on_view_back_button_clicked(b):
    showPage("Welcome")
    search_product_textbox.value = ''
    initialize_records()
  view_product_back_button.on_click(on_view_back_button_clicked)


  # Show product details
  def on_ret_5_detail_button_clicked(b):
    button_num = 0
    for button_num in range(len(ret_5_detail_buttons)):
      if ret_5_detail_buttons[button_num] == b:
        product_id_selected = ret_5_id_textboxes[button_num].value
        break

    dtl_id_textbox.value = str(product_id_selected)
    c.execute("SELECT * FROM PRODUCT WHERE P_ID = '{}'".format(product_id_selected))

    for row in c:
      dtl_name_textbox.value = row[1]
      dtl_id_textbox.value = str(row[0])
      dtl_price_textbox.value = str(row[2])
      dtl_description_textbox.value = row[3]
      dtl_color_textbox.value = row[4]
      dtl_capacity_textbox.value = str(row[5])
      dtl_category_textbox.value = str(row[9])

      if row[6] <= 0:
        dtl_stock_label.value = "❗ Not Available. Please contact service point."
        dtl_add_shopping_cart.disabled = True
      else:
        dtl_stock_label.value = "✅ Available"
        dtl_add_shopping_cart.disabled = False

    showPage("Show_Product_Detail")

  for dtl_btn in ret_5_detail_buttons:
    dtl_btn.on_click(on_ret_5_detail_button_clicked)


  def on_search_clicked(b):
    global start_idx, search_results, current_searcg_type, total_records
    keyword = search_product_textbox.value.strip()

    if not keyword:
      current_search_type = None
      search_results = []
      start_idx = 0
      initialize_records()
      return
    current_search_type = 'keyword'

    if keyword.isnumeric():
      c.execute("SELECT P_ID, P_NAME, P_CATEGORY, P_PRICE FROM PRODUCT WHERE LOWER(P_NAME) LIKE :1 OR P_ID = :2 ORDER BY P_ID",
              ('%' + keyword.lower() + '%', int(keyword),))
    else:
      c.execute("SELECT P_ID, P_NAME, P_CATEGORY, P_PRICE FROM PRODUCT WHERE LOWER(P_NAME) LIKE :1 ORDER BY P_ID",
              ('%' + keyword.lower() + '%',))

    search_results = c.fetchall()
    total_records = len(search_results)
    start_idx = 0
    show_records()
    search_product_textbox.value = ''
  search_button.on_click(on_search_clicked)


  def on_search_by_cat_clicked(b):
    global start_idx, search_results, current_search_type, total_records
    keyword = sort_by_category_dropdown.value

    if keyword == "All categories":
      current_search_type = None
      search_results = None
      start_id = 0
      initialize_records()
      sort_by_category_dropdown.value = None
      return
    elif not keyword:
      current_search_type = None
      search_results = None
      start_idx = 0
      initialize_records()
      sort_by_category_dropdown.value = None
      return

    current_search_type = 'category'
    c.execute("SELECT P_ID, P_NAME, P_CATEGORY, P_PRICE FROM PRODUCT WHERE P_CATEGORY = :category ORDER BY P_ID",{'category':keyword})
    search_results = c.fetchall()
    start_idx = 0
    total_records = len(search_results)
    show_records()
    sort_by_category_dropdown.value = None
  sortbycategory_button.on_click(on_search_by_cat_clicked)


  def on_search_by_price_clicked(b):
    global start_idx, search_results, current_search_type, total_records
    price_interval = sort_by_price_dropdown.value

    current_search_type = 'price'

    if price_interval == "All prices" or price_interval is None:
      c.execute("SELECT P_ID, P_NAME, P_CATEGORY, P_PRICE FROM PRODUCT ORDER BY P_ID, P_PRICE")

    else:
      min_price, max_price = price_interval
      c.execute("""
      SELECT P_ID, P_NAME, P_CATEGORY, P_PRICE FROM PRODUCT WHERE P_PRICE BETWEEN '{}' AND '{}' ORDER BY P_ID, P_PRICE
      """.format(min_price, max_price))

    search_results = c.fetchall()
    start_idx = 0
    total_records = len(search_results)
    show_records()
    sort_by_price_dropdown.value = None
  sortbyprice_button.on_click(on_search_by_price_clicked)

  initialize_records()

  ret_5_rcd_ctrl_HBox = widgets.HBox([previous_5_button, next_5_button])
  ret_5_VBox_items = [search_status_label] + ret_5_data_HBoxes + [ret_5_rcd_ctrl_HBox, view_product_back_button]

  ret_5_VBox = widgets.VBox(ret_5_VBox_items)

  search_box.children = [keyword_search_box,
              widgets.HTML("<hr style='margin: 10px 0; border: 1px solid #ddd;'>"),
              category_search_box,
              widgets.HTML(("<hr style='margin: 10px 0; border: 1px solid #ddd;'>")),
              price_search_box]

  display(widgets.VBox([search_box,ret_5_VBox]))

##############################################
##### Show Product Details Tab #########
##############################################

with tb_all.output_to("Show_Product_Detail"):
  dtl_id_label = Label("Product ID: ", layout ={'margin':'5px'})
  dtl_id_textbox = Text(value='', placeholder='Product ID', disabled=True, layout={'width': '200px', 'margin': '5px'})

  dtl_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

  dtl_name_label = Label("Product Name: ", layout ={'margin':'5px'})
  dtl_name_textbox = Text(placeholder='Product Name:', layout={'width': '200px', 'margin': '5px'}, disabled = True)

  dtl_price_label = Label("Price: ", layout ={'margin':'5px'})
  dtl_price_textbox = Text(placeholder='Price:', layout={'width': '200px', 'margin': '5px'},disabled = True)

  dtl_description_label = Label("Product Description: ", layout ={'margin':'5px'})
  dtl_description_textbox = Text(placeholder='Product Description:', layout={'width': '200px', 'margin': '5px'},disabled = True)

  dtl_color_label = Label("Color: ", layout ={'margin':'5px'})
  dtl_color_textbox = Text(placeholder='Color:', layout={'width': '200px', 'margin': '5px'},disabled = True)

  dtl_capacity_label = Label("Capacity: ", layout ={'margin':'5px'})
  dtl_capacity_textbox = Text(placeholder='Capacity:', layout={'width': '200px', 'margin': '5px'},disabled = True)

  dtl_category_label = Label("Category: ", layout ={'margin':'5px'})
  dtl_category_textbox = Text(placeholder='Category:', layout={'width': '200px', 'margin': '5px'},disabled = True)

  dtl_stock_label = Label(" ", layout={'width': '300px', 'margin': '5px'})

  dtl_item_count_label = Label("Purchase Number:", layout ={'margin':'5px'})
  dtl_item_count_intText = widgets.IntText(disabled=False,layout={'width': '200px', 'margin': '5px'})

  dtl_add_shopping_cart = widgets.Button(description = "Add to Shopping Cart", layout = {'width': '200px', 'margin': '5px'})
  dtl_back_button = widgets.Button(description = "Back")
  dtl_status_label = Label("")

  def on_dtl_back_button_clicked(b):
    showPage("Search_Product")
    show_records()
    dtl_status_label.value = ""
    dtl_item_count_intText.value = 0
  dtl_back_button.on_click(on_dtl_back_button_clicked)

  # Put product into shopping cart
  def on_dtl_add_shopping_cart_clicked(b):

    try:
      if not member_id.value:
        dtl_status_label.value = "❗ Please login first."
        return

      if dtl_item_count_intText.value <= 0:
        dtl_status_label.value = "❗ Quantity must be at least 1."
        return

      member_id_val = int(member_id.value)
      product_id = int(dtl_id_textbox.value)
      quantity = int(dtl_item_count_intText.value)
      current_date = datetime.now().strftime('%Y-%m-%d')


      c.execute(
      # SC_ID, SC_ITEM_COUNT, SC_LAST_UPDATE, MEMBER_M_ID, SC_STATUS, PRODUCT_P_ID
      """
      INSERT INTO SHOPPINGCART (SC_ID, SC_ITEM_COUNT, SC_LAST_UPDATE, SC_STATUS, MEMBER_M_ID, PRODUCT_P_ID)
      VALUES (seq_shoppingcart.nextval, :quantity, TO_DATE(:current_date,'YYYY-MM-DD'), 'saved', :member_id, :product_id)
      """,
      {
        'quantity': quantity,
        'current_date': current_date,
        'member_id': member_id_val,
        'product_id': product_id
      }
      )
      c.execute("commit")
      dtl_add_shopping_cart.disabled = True
      dtl_status_label.value = "✅ Successfully added."

    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      dtl_status_label.value = f"Database Error: {error_obj.message}"
    except ValueError as e:
      dtl_status_label.value = f"Invalid input: {str(e)}"


  dtl_add_shopping_cart.on_click(on_dtl_add_shopping_cart_clicked)


  # display boxes
  dtl_line1HBox = widgets.HBox([dtl_id_label, dtl_id_textbox, dtl_stock_label])
  dtl_line2HBox = widgets.HBox([dtl_spacer])
  dtl_labelVBox = widgets.VBox([dtl_name_label, dtl_price_label, dtl_description_label, dtl_color_label, dtl_capacity_label, dtl_category_label,dtl_item_count_label])
  dtl_textboxVBox = widgets.VBox([dtl_name_textbox, dtl_price_textbox, dtl_description_textbox, dtl_color_textbox, dtl_capacity_textbox, dtl_category_textbox, dtl_item_count_intText])
  dtl_line3HBox = widgets.HBox([dtl_labelVBox, dtl_textboxVBox])
  dtl_line4HBox = widgets.HBox([dtl_add_shopping_cart, dtl_back_button])

  display(widgets.VBox([dtl_line1HBox, dtl_line2HBox, dtl_line3HBox, dtl_status_label, dtl_line4HBox]))

##############################################
############## Shopping_Cart Tab ##############
##############################################

with tb_all.output_to('Shopping Cart'):
  # Create styled title
  cart_title = Label("Your Shopping Cart")
  cart_title.style.font_weight = 'bold'
  cart_title.style.font_size = '18px'
  cart_status = Label("")

  # Create lists for cart items display
  cart_item_ids = []
  cart_sc_ids = []
  cart_product_names = []
  cart_quantities = []
  cart_prices = []
  cart_subtotals = []
  cart_edit_buttons = []
  cart_remove_buttons = []
  cart_item_rows = []

  # Create container for cart items
  cart_items_output = widgets.Output()

  # Total price display with styling
  total_price_label = Label("Total: $0.00")
  total_price_label.style.font_weight = 'bold'

  # Styled buttons
  checkout_button = widgets.Button(description="Proceed to Checkout",
                                  button_style='success',
                                  layout=widgets.Layout(width='180px'))
  back_button = widgets.Button(description="Back",
                              button_style='info',
                              layout=widgets.Layout(width='100px'))
  refresh_button = widgets.Button(description="Refresh Cart",
                                 button_style='primary',
                                 layout=widgets.Layout(width='150px'))

  # Function to show cart items
  def show_cart_items():
    # Clear previous items
    cart_item_ids.clear()
    cart_sc_ids.clear()
    cart_product_names.clear()
    cart_quantities.clear()
    cart_prices.clear()
    cart_subtotals.clear()
    cart_edit_buttons.clear()
    cart_remove_buttons.clear()
    cart_item_rows.clear()

    cart_items_output.clear_output()

    try:
      # Get cart items for the member
      c.execute("""
        SELECT sc.SC_ID, p.P_ID, p.P_NAME, sc.SC_ITEM_COUNT, p.P_PRICE, (sc.SC_ITEM_COUNT * p.P_PRICE) as SUBTOTAL
        FROM SHOPPINGCART sc
        JOIN PRODUCT p ON sc.PRODUCT_P_ID = p.P_ID
        WHERE sc.MEMBER_M_ID = :member_id
        AND sc.SC_STATUS = 'saved'
        AND p.P_STOCK_QTY > 0
      """, {'member_id': member_id.value})

      cart_items = c.fetchall()

      if not cart_items:
        with cart_items_output:
          display(Label("Your shopping cart is empty."))
        total_price_label.value = "Total: $0.00"
        checkout_button.disabled = True
        return

      # Create styled header row
      header_row = widgets.HBox([
        Label("Product ID", layout={'width': '120px'}),
        Label("Product", layout={'width': '250px'}),
        Label("Quantity", layout={'width': '120px'}),
        Label("Price", layout={'width': '120px'}),
        Label("Subtotal", layout={'width': '150px'}),
        Label("", layout={'width': '80px'}),  # For edit button
        Label("", layout={'width': '80px'})   # For remove button
      ])

      # Style the header row
      for child in header_row.children:
        if isinstance(child, Label):
          child.style.font_weight = 'bold'

      item_rows = [header_row]

      total_price = 0

      for item in cart_items:
        sc_id = item[0]
        product_id = item[1]
        product_name = item[2]
        quantity = item[3]
        price = item[4]
        subtotal = item[5]

        total_price += subtotal

        # Create widgets for this item with consistent styling
        cart_sc_ids.append(Text(value=str(sc_id), disabled=True,
                               layout=widgets.Layout(display='none')))

        cart_item_ids.append(Text(value=str(product_id), disabled=True,
                                 layout=widgets.Layout(width='120px', margin='0px 5px 0px 0px')))

        cart_product_names.append(Text(value=product_name, disabled=True,
                                      layout=widgets.Layout(width='250px', margin='0px 5px 0px 0px')))

        cart_quantities.append(IntText(value=quantity, min=1,
                                      layout=widgets.Layout(width='120px', margin='0px 5px 0px 0px')))

        cart_prices.append(Text(value=f"${price:.2f}", disabled=True,
                               layout=widgets.Layout(width='120px', margin='0px 5px 0px 0px')))

        cart_subtotals.append(Text(value=f"${subtotal:.2f}", disabled=True,
                                  layout=widgets.Layout(width='150px', margin='0px 5px 0px 0px')))

        # Create styled edit and remove buttons for this item
        edit_button = widgets.Button(description="Update",
                                    button_style='primary',
                                    layout=widgets.Layout(width='80px', margin='0px 5px 0px 0px'))

        remove_button = widgets.Button(description="Remove",
                                      button_style='danger',
                                      layout=widgets.Layout(width='80px'))

        cart_edit_buttons.append(edit_button)
        cart_remove_buttons.append(remove_button)

        # Define button click handlers
        def create_edit_handler(index):
          def edit_handler(b):
            update_cart_item(index)
          return edit_handler

        def create_remove_handler(index):
          def remove_handler(b):
            remove_cart_item(index)
          return remove_handler

        edit_button.on_click(create_edit_handler(len(cart_edit_buttons) - 1))
        remove_button.on_click(create_remove_handler(len(cart_remove_buttons) - 1))

        # Create styled row for this item
        item_row = widgets.HBox([
          cart_item_ids[-1],
          cart_product_names[-1],
          cart_quantities[-1],
          cart_prices[-1],
          cart_subtotals[-1],
          edit_button,
          remove_button
        ], layout=widgets.Layout(margin='5px 0px'))

        item_rows.append(item_row)

      # Display all items in a styled container
      with cart_items_output:
        display(widgets.VBox(item_rows,
                            layout=widgets.Layout(
                                border='1px solid #ddd',
                                padding='10px',
                                margin='10px 0px',
                                border_radius='5px')))

      total_price_label.value = f"Total: ${total_price:.2f}"
      checkout_button.disabled = False

    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      cart_status.value = f"Database Error: {error_obj.message}"
      checkout_button.disabled = True

  # Function to update cart item
  def update_cart_item(index):
    try:
      sc_id = cart_sc_ids[index].value
      new_quantity = cart_quantities[index].value

      if new_quantity <= 0:
        cart_status.value = "❗ Quantity must be at least 1"
        return

      # Update quantity in database
      c.execute("""
        UPDATE SHOPPINGCART
        SET SC_ITEM_COUNT = :quantity,
            SC_LAST_UPDATE = SYSDATE
        WHERE SC_ID = :sc_id
      """, {'quantity': new_quantity, 'sc_id': sc_id})

      c.execute("COMMIT")

      cart_status.value = "✅ Cart item updated successfully"
      show_cart_items()  # Refresh the cart

    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      cart_status.value = f"Database Error: {error_obj.message}"

  # Function to remove cart item
  def remove_cart_item(index):
    try:
      sc_id = cart_sc_ids[index].value

      # Remove item from database
      c.execute("""
        DELETE FROM SHOPPINGCART
        WHERE SC_ID = :sc_id
      """, {'sc_id': sc_id})

      c.execute("COMMIT")

      cart_status.value = "✅ Cart item removed successfully"
      show_cart_items()  # Refresh the cart

    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      cart_status.value = f"Database Error: {error_obj.message}"

  # Button click handlers
  def on_back_button_clicked(b):
    showPage("Welcome")

  def on_refresh_button_clicked(b):
    show_cart_items()

  def on_checkout_button_clicked(b):
    showPage("Checkout")

  back_button.on_click(on_back_button_clicked)
  refresh_button.on_click(on_refresh_button_clicked)
  checkout_button.on_click(on_checkout_button_clicked)

  # Styled layout
  cart_header = widgets.HBox([cart_title],layout=widgets.Layout(justify_content='center',margin='10px 0px 20px 0px'))

  cart_buttons = widgets.HBox([refresh_button, back_button],layout=widgets.Layout(justify_content='space-between',margin='10px 0px'))

  cart_footer = widgets.HBox([total_price_label, checkout_button],layout=widgets.Layout(justify_content='space-between',margin='15px 0px'))

  # Main layout
  display(widgets.VBox([
    cart_header,
    cart_buttons,
    cart_items_output,
    cart_status,
    cart_footer
  ], layout=widgets.Layout(
      padding='15px',
      margin='10px')))

##############################################
############## Checkout Tab ##############
##############################################

with tb_all.output_to('Checkout'):
  # Create styled title
  checkout_title = Label("Checkout")
  checkout_title.style.font_weight = 'bold'
  checkout_title.style.font_size = '18px'
  checkout_status = Label("")

  # Order summary section with styling
  order_summary_label = Label("Order Summary")
  order_summary_label.style.font_weight = 'bold'
  order_summary_label.style.font_size = '16px'

  order_items_output = widgets.Output()

  # Total price display with styling
  order_total_label = Label("Total: $0.00")
  order_total_label.style.font_weight = 'bold'

  # Payment information section with styling
  payment_label = Label("Payment Information")
  payment_label.style.font_weight = 'bold'
  payment_label.style.font_size = '16px'

  # Credit card information with consistent styling
  credit_card_number_label = Label("Credit Card Number:", layout=widgets.Layout(width='150px'))
  credit_card_number_textbox = Text(placeholder='Enter your credit card number',
                                   disabled=False,
                                   layout=widgets.Layout(width='250px'))

  expiry_date_label = Label("Expiry Date:", layout=widgets.Layout(width='150px'))
  expiry_date_picker = DatePicker(disabled=False,
                                 layout=widgets.Layout(width='250px'))

  cvc_label = Label("CVC Code:", layout=widgets.Layout(width='150px'))
  cvc_textbox = Text(placeholder='Enter CVC code',
                    disabled=False,
                    layout=widgets.Layout(width='250px'))

  # Trade-in option section with styling
  trade_in_label = Label("Trade-In Options")
  trade_in_label.style.font_weight = 'bold'
  trade_in_label.style.font_size = '16px'

  trade_in_checkbox = widgets.Checkbox(
    value=False,
    description='I want to trade in old device(s)',
    disabled=False
  )

  # Trade-in form (initially hidden)
  trade_in_form_output = widgets.Output()

  # Trade-in device information with consistent styling
  device_type_label = Label("Device Type:", layout=widgets.Layout(width='150px'))
  device_type_dropdown = widgets.Dropdown(
    options=['Smartphone', 'Tablet', 'Smartwatch', 'Laptop'],
    value='Smartphone',
    disabled=False,
    layout=widgets.Layout(width='250px')
  )

  device_model_label = Label("Device Model:", layout=widgets.Layout(width='150px'))
  device_model_textbox = Text(placeholder='e.g., iPhone 12, Galaxy S21',
                             disabled=False,
                             layout=widgets.Layout(width='250px'))

  device_serial_label = Label("Serial Number:", layout=widgets.Layout(width='150px'))
  device_serial_textbox = Text(placeholder='Enter device serial number',
                              disabled=False,
                              layout=widgets.Layout(width='250px'))

  device_condition_label = Label("Device Condition:", layout=widgets.Layout(width='150px'))
  device_condition_dropdown = widgets.Dropdown(
    options=['Excellent', 'Good', 'Fair', 'Poor'],
    value='Good',
    disabled=False,
    layout=widgets.Layout(width='250px')
  )

  device_description_label = Label("Additional Description:", layout=widgets.Layout(width='150px'))
  device_description_textbox = Text(placeholder='Any additional information about the device',
                                   disabled=False,
                                   layout=widgets.Layout(width='250px'))

  # Styled button for adding devices
  add_another_device_button = widgets.Button(description="Add Device",
                                            button_style='primary',
                                            layout=widgets.Layout(width='180px'))

  # List to store trade-in devices
  trade_in_devices = []
  trade_in_devices_output = widgets.Output()

  # Styled buttons for checkout actions
  confirm_order_button = widgets.Button(description="Confirm Order",
                                       button_style='success',
                                       layout=widgets.Layout(width='150px'))

  back_to_cart_button = widgets.Button(description="Back to Cart",
                                      button_style='info',
                                      layout=widgets.Layout(width='150px'))

  # Function to prepare checkout page
  def prepare_checkout():
    # Debug message to confirm function is called
    checkout_status.value = "Loading checkout page..."

    # Clear all outputs
    order_items_output.clear_output()
    trade_in_form_output.clear_output()
    trade_in_devices_output.clear_output()
    trade_in_devices.clear()

    # Reset form fields
    credit_card_number_textbox.value = ""
    cvc_textbox.value = ""
    expiry_date_picker.value = None
    trade_in_checkbox.value = False
    device_model_textbox.value = ""
    device_serial_textbox.value = ""
    device_description_textbox.value = ""

    try:
      # Get cart items for the member
      c.execute("""
        SELECT p.P_NAME, sc.SC_ITEM_COUNT, p.P_PRICE, (sc.SC_ITEM_COUNT * p.P_PRICE) as SUBTOTAL
        FROM SHOPPINGCART sc
        JOIN PRODUCT p ON sc.PRODUCT_P_ID = p.P_ID
        WHERE sc.MEMBER_M_ID = :member_id
        AND sc.SC_STATUS = 'saved'
        AND p.P_STOCK_QTY > 0
        ORDER BY p.P_ID
      """, {'member_id': member_id.value})

      cart_items = c.fetchall()

      if not cart_items:
        with order_items_output:
          display(Label("Your shopping cart is empty."))
        order_total_label.value = "Total: $0.00"
        confirm_order_button.disabled = True
        return

      total_price = 0
      order_rows = []

      # Create styled header row
      header_row = widgets.HBox([
        Label("Product", layout={'width': '250px'}),
        Label("Quantity", layout={'width': '100px'}),
        Label("Price", layout={'width': '100px'}),
        Label("Subtotal", layout={'width': '100px'})
      ])

      # Style the header row
      for child in header_row.children:
        if isinstance(child, Label):
          child.style.font_weight = 'bold'

      order_rows.append(header_row)

      for item in cart_items:
        product_name = item[0]
        quantity = item[1]
        price = item[2]
        subtotal = item[3]

        total_price += subtotal

        # Create styled row for this item
        item_row = widgets.HBox([
          Label(product_name, layout={'width': '250px'}),
          Label(str(quantity), layout={'width': '100px'}),
          Label(f"${price:.2f}", layout={'width': '100px'}),
          Label(f"${subtotal:.2f}", layout={'width': '100px'})
        ], layout=widgets.Layout(margin='5px 0px'))

        order_rows.append(item_row)

      # Add styled total row
      total_row = widgets.HBox([
        Label("", layout={'width': '250px'}),
        Label("", layout={'width': '100px'}),
        Label("Total:", layout={'width': '100px', 'font_weight': 'bold'}),
        Label(f"${total_price:.2f}", layout={'width': '100px', 'font_weight': 'bold'})
      ], layout=widgets.Layout(margin='10px 0px 5px 0px'))

      order_rows.append(total_row)

      # Use a completely different approach with direct widget display
      order_items_output.clear_output()

      # Calculate total price
      total_price = 0
      for item in cart_items:
        subtotal = item[3]
        total_price += subtotal

      # Create a list of widgets to display the order items
      with order_items_output:
        # Create a header with bold styling
        header = widgets.HBox([
          widgets.HTML("<b>Product</b>", layout=widgets.Layout(width='250px')),
          widgets.HTML("<b>Quantity</b>", layout=widgets.Layout(width='100px')),
          widgets.HTML("<b>Price</b>", layout=widgets.Layout(width='100px')),
          widgets.HTML("<b>Subtotal</b>", layout=widgets.Layout(width='100px'))
        ], layout=widgets.Layout(margin='5px 0px', border='1px solid #ddd', padding='5px', background_color='#f2f2f2'))

        display(header)

        # Add each product row
        for item in cart_items:
          product_name = item[0]
          quantity = item[1]
          price = item[2]
          subtotal = item[3]

          row = widgets.HBox([
            widgets.Label(product_name, layout=widgets.Layout(width='250px')),
            widgets.Label(str(quantity), layout=widgets.Layout(width='100px')),
            widgets.Label(f"${price:.2f}", layout=widgets.Layout(width='100px')),
            widgets.Label(f"${subtotal:.2f}", layout=widgets.Layout(width='100px'))
          ], layout=widgets.Layout(margin='5px 0px', border='1px solid #ddd', padding='5px'))

          display(row)

        # Add total row with bold styling
        total_row = widgets.HBox([
          widgets.Label("", layout=widgets.Layout(width='250px')),
          widgets.Label("", layout=widgets.Layout(width='100px')),
          widgets.HTML("<b>Total:</b>", layout=widgets.Layout(width='100px')),
          widgets.HTML(f"<b>${total_price:.2f}</b>", layout=widgets.Layout(width='100px'))
        ], layout=widgets.Layout(margin='5px 0px', border='1px solid #ddd', padding='5px', background_color='#f9f9f9'))

        display(total_row)

      # Make sure the output is visible
      order_items_output.layout.visibility = 'visible'
      order_items_output.layout.display = 'block'

      # Update the total label
      order_total_label.value = f"Total: ${total_price:.2f}"

      # Debug message to confirm items are displayed
      checkout_status.value = f"Order summary loaded with {len(cart_items)} items. Total: ${total_price:.2f}"

      order_total_label.value = f"Total: ${total_price:.2f}"
      confirm_order_button.disabled = False

      # Setup trade-in form with styling
      with trade_in_form_output:
        trade_in_form_label = Label("Trade-In Device Information")
        trade_in_form_label.style.font_weight = 'bold'

        device_type_row = widgets.HBox([device_type_label, device_type_dropdown],
                                      layout=widgets.Layout(margin='5px 0px'))

        device_model_row = widgets.HBox([device_model_label, device_model_textbox],
                                       layout=widgets.Layout(margin='5px 0px'))

        device_serial_row = widgets.HBox([device_serial_label, device_serial_textbox],
                                        layout=widgets.Layout(margin='5px 0px'))

        device_condition_row = widgets.HBox([device_condition_label, device_condition_dropdown],
                                          layout=widgets.Layout(margin='5px 0px'))

        device_description_row = widgets.HBox([device_description_label, device_description_textbox],
                                            layout=widgets.Layout(margin='5px 0px'))

        add_device_button_row = widgets.HBox([add_another_device_button],
                                           layout=widgets.Layout(margin='15px 0px'))

        # Display trade-in form in a styled container
        display(widgets.VBox([
          trade_in_form_label,
          device_type_row,
          device_model_row,
          device_serial_row,
          device_condition_row,
          device_description_row,
          add_device_button_row,
          trade_in_devices_output
        ], layout=widgets.Layout(
            border='1px solid #ddd',
            padding='15px',
            margin='10px 0px',
            border_radius='5px')))

    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      checkout_status.value = f"Database Error: {error_obj.message}"
      confirm_order_button.disabled = True

  # Function to handle trade-in checkbox
  def on_trade_in_checkbox_change(change):
    if change['new']:
      # Show the trade-in form
      trade_in_form_output.clear_output()
      with trade_in_form_output:
        trade_in_form_label = Label("Trade-In Device Information")
        trade_in_form_label.style.font_weight = 'bold'

        device_type_row = widgets.HBox([device_type_label, device_type_dropdown],
                                      layout=widgets.Layout(margin='5px 0px'))

        device_model_row = widgets.HBox([device_model_label, device_model_textbox],
                                       layout=widgets.Layout(margin='5px 0px'))

        device_serial_row = widgets.HBox([device_serial_label, device_serial_textbox],
                                        layout=widgets.Layout(margin='5px 0px'))

        device_condition_row = widgets.HBox([device_condition_label, device_condition_dropdown],
                                          layout=widgets.Layout(margin='5px 0px'))

        device_description_row = widgets.HBox([device_description_label, device_description_textbox],
                                            layout=widgets.Layout(margin='5px 0px'))

        add_device_button_row = widgets.HBox([add_another_device_button],
                                           layout=widgets.Layout(margin='15px 0px'))

        # Display trade-in form in a styled container
        display(widgets.VBox([
          trade_in_form_label,
          device_type_row,
          device_model_row,
          device_serial_row,
          device_condition_row,
          device_description_row,
          add_device_button_row,
          trade_in_devices_output
        ], layout=widgets.Layout(
            border='1px solid #ddd',
            padding='15px',
            margin='10px 0px',
            border_radius='5px')))

      # Make sure the form is visible
      trade_in_form_output.layout.display = 'block'
    else:
      # Hide the trade-in form
      trade_in_form_output.layout.display = 'none'
      trade_in_devices.clear()
      trade_in_devices_output.clear_output()

  trade_in_checkbox.observe(on_trade_in_checkbox_change, names='value')

  # Function to add trade-in device
  def on_add_another_device_button_clicked(b):
    # Validate inputs
    if device_model_textbox.value.strip() == "":
      checkout_status.value = "❗ Please enter device model"
      return

    if device_serial_textbox.value.strip() == "":
      checkout_status.value = "❗ Please enter device serial number"
      return

    if device_description_textbox.value.strip() == "":
      checkout_status.value = "❗ Please enter device description"
      return

    # Add device to list
    device = {
      'type': device_type_dropdown.value,
      'model': device_model_textbox.value,
      'serial': device_serial_textbox.value,
      'condition': device_condition_dropdown.value,
      'description': device_description_textbox.value
    }

    trade_in_devices.append(device)

    # Update display
    update_trade_in_devices_display()

    # Clear form for next device
    device_model_textbox.value = ""
    device_serial_textbox.value = ""
    device_description_textbox.value = ""

    checkout_status.value = "✅ Device added successfully! You can add another device or proceed with the order."

  # Function to update trade-in devices display
  def update_trade_in_devices_display():
    trade_in_devices_output.clear_output()

    with trade_in_devices_output:
      if trade_in_devices:
        devices_label = Label("Added Trade-In Devices:")
        devices_label.style.font_weight = 'bold'
        display(devices_label)

        # Create a styled list of added devices
        device_rows = []

        for i, device in enumerate(trade_in_devices):
          device_info = f"{i+1}. {device['model']} ({device['type']}) - {device['condition']} condition - SN: {device['serial']}"
          device_row = Label(device_info)
          device_rows.append(device_row)

        # Display devices in a styled container
        display(widgets.VBox(device_rows,
                            layout=widgets.Layout(
                                border='1px solid #ddd',
                                padding='10px',
                                margin='10px 0px',
                                border_radius='5px')))

  # Button click handlers
  def on_back_to_cart_button_clicked(b):
    showPage("Shopping Cart")

  def on_confirm_order_button_clicked(b):
    # Validate credit card information
    if credit_card_number_textbox.value.strip() == "":
      checkout_status.value = "❗ Please enter credit card number"
      return

    if not expiry_date_picker.value:
      checkout_status.value = "❗ Please select expiry date"
      return

    if cvc_textbox.value.strip() == "":
      checkout_status.value = "❗ Please enter CVC code"
      return

    # Validate credit card number format
    if not re.match(r"^[0-9]{8,20}$", credit_card_number_textbox.value):
      checkout_status.value = "❗ Credit card number must be 8-20 digits"
      return

    # Validate CVC format
    if not re.match(r"^[0-9]{3,4}$", cvc_textbox.value):
      checkout_status.value = "❗ CVC must be 3-4 digits"
      return

    # Create order
    create_order()

  # Function to create order
  def create_order():
    try:
      # Create order in database - using sequence directly in SQL
      # Provide a default value for OR_REMARK to avoid NULL constraint error
      c.execute("""
        INSERT INTO ORDERS (OR_ID, OR_STATUS, OR_DATE, OR_PAY_METHOD, OR_DISCOUNT, OR_SHIPDATE, OR_ESTIMATED_DAY, OR_REMARK, MEMBER_M_ID)
        VALUES (seq_orders.nextval, 'Pending', SYSDATE, 'Credit Card', 0, SYSDATE+7, SYSDATE+14, 'Order placed online', :member_id)
      """, {
        'member_id': member_id.value
      })

      # Get the new order ID using currval
      c.execute("SELECT seq_orders.currval FROM dual")
      new_order_id = c.fetchone()[0]
      order_id.value = str(new_order_id)

      # Get cart items
      c.execute("""
        SELECT sc.PRODUCT_P_ID, p.P_PRICE, sc.SC_ITEM_COUNT, (sc.SC_ITEM_COUNT * p.P_PRICE) as SUBTOTAL
        FROM SHOPPINGCART sc
        JOIN PRODUCT p ON sc.PRODUCT_P_ID = p.P_ID
        WHERE sc.MEMBER_M_ID = :member_id
        AND sc.SC_STATUS = 'saved'
        AND p.P_STOCK_QTY > 0
      """, {'member_id': member_id.value})

      cart_items = c.fetchall()

      # Create order lines for each cart item
      for item in cart_items:
        product_id = item[0]
        price = item[1]
        quantity = item[2]
        subtotal = item[3]

        c.execute("""
          INSERT INTO ORDERLINE (OL_ID, OL_TOTALPRICE, OL_SUBTOTAL, OL_P_QTY, ORDERS_OR_ID, PRODUCT_P_ID)
          VALUES (seq_orderline.nextval, :total_price, :subtotal, :quantity, :order_id, :product_id)
        """, {
          'total_price': subtotal,
          'subtotal': subtotal,
          'quantity': quantity,
          'order_id': new_order_id,
          'product_id': product_id
        })

        # Update product stock
        c.execute("""
          UPDATE PRODUCT
          SET P_STOCK_QTY = P_STOCK_QTY - :quantity
          WHERE P_ID = :product_id
        """, {
          'quantity': quantity,
          'product_id': product_id
        })

      # Add trade-in information if applicable
      if trade_in_checkbox.value and trade_in_devices:
        # Update order status to indicate trade-in
        c.execute("""
          UPDATE ORDERS
          SET OR_STATUS = 'Pending Trade-In',
              OR_REMARK = 'Trade-in requested'
          WHERE OR_ID = :order_id
        """, {'order_id': new_order_id})

        # Add each trade-in device
        for device in trade_in_devices:
          c.execute("""
            INSERT INTO TRADE_IN (TR_ID, TR_ITEM, TR_SERIAL_NUMBER, TR_ITEM_DECSRIPTION, TR_PRICE, TR_DATE, TR_ITEM_CONDITION, TR_STATUS, ORDERS_OR_ID)
            VALUES (seq_trade_in.nextval, :item, :serial, :description, 0, SYSDATE, :condition, 'Pending', :order_id)
          """, {
            'item': device['model'],
            'serial': device['serial'],
            'description': device['description'],
            'condition': device['condition'],
            'order_id': new_order_id
          })


      # Clear shopping cart
      c.execute("""
        DELETE FROM SHOPPINGCART
        WHERE MEMBER_M_ID = :member_id
        AND SC_STATUS = 'saved'
      """, {'member_id': member_id.value})

      c.execute("COMMIT")

      # Display success message with styling
      if trade_in_checkbox.value and trade_in_devices:
        checkout_status.value = f"✅ Order placed successfully with trade-in request! Order ID: {new_order_id}"
      else:
        checkout_status.value = f"✅ Order placed successfully! Order ID: {new_order_id}"

      confirm_order_button.disabled = True

    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      checkout_status.value = f"Database Error: {error_obj.message}"
      c.execute("ROLLBACK")

  add_another_device_button.on_click(on_add_another_device_button_clicked)
  back_to_cart_button.on_click(on_back_to_cart_button_clicked)
  confirm_order_button.on_click(on_confirm_order_button_clicked)

  # Set initial state of trade-in form
  trade_in_form_output.layout.display = 'none'

  # Styled layout
  checkout_header = widgets.HBox([checkout_title],
                                layout=widgets.Layout(justify_content='center',
                                                     margin='10px 0px 20px 0px'))

  # Order summary section with styling
  order_summary_header = widgets.HBox([order_summary_label],
                                     layout=widgets.Layout(margin='15px 0px 10px 0px'))

  order_summary_footer = widgets.HBox([order_total_label],
                                     layout=widgets.Layout(margin='10px 0px'))

  # Payment section with styling
  payment_section_header = widgets.HBox([payment_label],
                                       layout=widgets.Layout(margin='15px 0px 10px 0px'))

  # Credit card information rows with consistent styling
  credit_card_row = widgets.HBox([credit_card_number_label, credit_card_number_textbox],
                                layout=widgets.Layout(margin='5px 0px'))

  expiry_date_row = widgets.HBox([expiry_date_label, expiry_date_picker],
                                layout=widgets.Layout(margin='5px 0px'))

  cvc_row = widgets.HBox([cvc_label, cvc_textbox],
                        layout=widgets.Layout(margin='5px 0px'))

  # Payment info container
  payment_info_container = widgets.VBox([
    payment_section_header,
    credit_card_row,
    expiry_date_row,
    cvc_row
  ], layout=widgets.Layout(
      border='1px solid #ddd',
      padding='10px',
      margin='10px 0px',
      border_radius='5px'))

  # Trade-in section with styling
  trade_in_section_header = widgets.HBox([trade_in_label],
                                        layout=widgets.Layout(margin='15px 0px 10px 0px'))

  trade_in_section = widgets.VBox([trade_in_checkbox],
                                 layout=widgets.Layout(margin='5px 0px'))

  # Trade-in container
  trade_in_container = widgets.VBox([
    trade_in_section_header,
    trade_in_section
  ], layout=widgets.Layout(
      border='1px solid #ddd',
      padding='10px',
      margin='10px 0px',
      border_radius='5px'))

  # Button row with styling
  buttons_section = widgets.HBox([back_to_cart_button, confirm_order_button],
                                layout=widgets.Layout(justify_content='space-between',
                                                     margin='15px 0px'))

  # Create a direct container for order items to ensure visibility
  order_summary_container = widgets.VBox([
    order_summary_header,
    order_items_output,
    order_summary_footer
  ], layout=widgets.Layout(
      border='2px solid #28a745',  # Green border to make it stand out
      padding='10px',
      margin='10px 0px',
      border_radius='5px'))

  # Main layout
  display(widgets.VBox([
    checkout_header,
    order_summary_container,
    payment_info_container,
    trade_in_container,
    trade_in_form_output,
    checkout_status,
    buttons_section
  ], layout=widgets.Layout(
      padding='15px',
      margin='10px')))

##############################################
############## Order_List Tab ################
##############################################

with tb_all.output_to('Order List'):
  # Create styled title
  order_list_title = Label("Your Orders")
  order_list_title.style.font_weight = 'bold'
  order_list_title.style.font_size = '18px'
  order_list_status = Label("")

  # Create lists for order display
  order_ids = []
  order_dates = []
  order_statuses = []
  order_totals = []
  order_detail_buttons = []
  order_rows = []

  # Create container for orders
  orders_output = widgets.Output()

  # Styled buttons
  back_button = widgets.Button(description="Back",
                              button_style='info',
                              layout=widgets.Layout(width='100px'))
  refresh_button = widgets.Button(description="Refresh Orders",
                                 button_style='primary',
                                 layout=widgets.Layout(width='150px'))

  # Function to show orders
  def show_orders():
    # Clear previous items
    order_ids.clear()
    order_dates.clear()
    order_statuses.clear()
    order_totals.clear()
    order_detail_buttons.clear()
    order_rows.clear()

    orders_output.clear_output()

    try:
      # Get orders for the member
      c.execute("""
        SELECT o.OR_ID, o.OR_DATE, o.OR_STATUS,
               NVL((SELECT SUM(ol.OL_TOTALPRICE) FROM ORDERLINE ol WHERE ol.ORDERS_OR_ID = o.OR_ID), 0) as TOTAL_PRICE
        FROM ORDERS o
        WHERE o.MEMBER_M_ID = :member_id
        ORDER BY o.OR_DATE DESC
      """, {'member_id': member_id.value})

      orders = c.fetchall()

      if not orders:
        with orders_output:
          display(Label("You have no orders."))
        return

      # Create styled header row
      header_row = widgets.HBox([
        Label("Order ID", layout={'width': '100px'}),
        Label("Date", layout={'width': '150px'}),
        Label("Status", layout={'width': '150px'}),
        Label("Total", layout={'width': '100px'}),
        Label("", layout={'width': '100px'})  # For button
      ])

      # Style the header row
      for child in header_row.children:
        if isinstance(child, Label):
          child.style.font_weight = 'bold'

      order_rows.append(header_row)

      for order in orders:
        order_id_val = order[0]
        order_date = order[1]
        order_status = order[2]
        order_total = order[3]

        # Format date
        formatted_date = order_date.strftime("%Y-%m-%d")

        # Create widgets for this order with consistent styling
        id_text = Text(value=str(order_id_val), disabled=True,
                      layout=widgets.Layout(width='100px', margin='0px 5px 0px 0px'))
        id_text.add_class('order-id')

        date_text = Text(value=formatted_date, disabled=True,
                        layout=widgets.Layout(width='150px', margin='0px 5px 0px 0px'))

        status_text = Text(value=order_status, disabled=True,
                          layout=widgets.Layout(width='150px', margin='0px 5px 0px 0px'))

        total_text = Text(value=f"${order_total:.2f}", disabled=True,
                         layout=widgets.Layout(width='100px', margin='0px 5px 0px 0px'))

        order_ids.append(id_text)
        order_dates.append(date_text)
        order_statuses.append(status_text)
        order_totals.append(total_text)

        # Create styled detail button for this order
        detail_button = widgets.Button(description="Details",
                                      button_style='primary',
                                      layout=widgets.Layout(width='100px'))
        order_detail_buttons.append(detail_button)

        # Define button click handler
        def create_detail_handler(index, order_id_val):
          def detail_handler(b):
            selected_order_id.value = str(order_id_val)
            showPage("Order Detail")
          return detail_handler

        detail_button.on_click(create_detail_handler(len(order_detail_buttons) - 1, order_id_val))

        # Create styled row for this order
        order_row = widgets.HBox([
          order_ids[-1],
          order_dates[-1],
          order_statuses[-1],
          order_totals[-1],
          detail_button
        ], layout=widgets.Layout(margin='5px 0px'))

        order_rows.append(order_row)

      # Display all orders in a styled container
      with orders_output:
        display(widgets.VBox(order_rows,
                            layout=widgets.Layout(
                                border='1px solid #ddd',
                                padding='10px',
                                margin='10px 0px',
                                border_radius='5px')))

    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      order_list_status.value = f"Database Error: {error_obj.message}"

  # Button click handlers
  def on_back_button_clicked(b):
    showPage("Welcome")

  def on_refresh_button_clicked(b):
    show_orders()

  back_button.on_click(on_back_button_clicked)
  refresh_button.on_click(on_refresh_button_clicked)

  # Styled layout
  order_list_header = widgets.HBox([order_list_title],
                                  layout=widgets.Layout(justify_content='center',
                                                       margin='10px 0px 20px 0px'))
  order_list_buttons = widgets.HBox([refresh_button, back_button],
                                   layout=widgets.Layout(justify_content='space-between',
                                                        margin='10px 0px'))

  display(widgets.VBox([
    order_list_header,
    order_list_buttons,
    orders_output,
    order_list_status
  ], layout=widgets.Layout(
      padding='15px',
      margin='10px')))

##############################################
############## Order_Detail Tab ##############
##############################################

with tb_all.output_to('Order Detail'):
  # Create styled title
  order_detail_title = Label("Order Detail")
  order_detail_title.style.font_weight = 'bold'
  order_detail_title.style.font_size = '18px'
  order_detail_status = Label("")

  # Order information section with styling
  order_info_label = Label("Order Information")
  order_info_label.style.font_weight = 'bold'
  order_info_label.style.font_size = '16px'

  # Order details with consistent styling
  order_id_label = Label("Order ID: ", layout=widgets.Layout(width='150px'))
  order_id_display = widgets.HTML("", layout=widgets.Layout(width='250px'))

  order_date_label = Label("Order Date: ", layout=widgets.Layout(width='150px'))
  order_date_display = Label("", layout=widgets.Layout(width='250px'))

  order_status_label = Label("Status: ", layout=widgets.Layout(width='150px'))
  order_status_display = Label("", layout=widgets.Layout(width='250px'))

  order_payment_label = Label("Payment Method: ", layout=widgets.Layout(width='150px'))
  order_payment_display = Label("", layout=widgets.Layout(width='250px'))

  # Order items section with styling
  order_items_label = Label("Order Items")
  order_items_label.style.font_weight = 'bold'
  order_items_label.style.font_size = '16px'
  order_items_output = widgets.Output()

  # Trade-in information section with styling
  trade_in_label = Label("Trade-In Information")
  trade_in_label.style.font_weight = 'bold'
  trade_in_label.style.font_size = '16px'

  trade_in_status_label = Label("Trade-In Status: ", layout=widgets.Layout(width='150px'))
  trade_in_status_display = Label("", layout=widgets.Layout(width='250px'))

  # Styled button
  view_trade_in_button = widgets.Button(description="View Trade-In Details",
                                       button_style='primary',
                                       disabled=True,
                                       layout=widgets.Layout(width='200px'))

  # Styled back button
  back_to_orders_button = widgets.Button(description="Back to Orders",
                                        button_style='info',
                                        layout=widgets.Layout(width='150px'))

  # Function to show order detail
  def show_order_detail():
    order_items_output.clear_output()

    try:
      # Get order information
      c.execute("""
        SELECT o.OR_ID, o.OR_DATE, o.OR_STATUS, o.OR_PAY_METHOD, o.OR_REMARK
        FROM ORDERS o
        WHERE o.OR_ID = :order_id
      """, {'order_id': selected_order_id.value})

      order = c.fetchone()

      if not order:
        order_detail_status.value = "❗ Order not found"
        return

      # Display order information with HTML formatting for Order ID
      order_id_display.value = f"<b>{order[0]}</b>"  # Bold HTML formatting
      order_date_display.value = order[1].strftime("%Y-%m-%d")
      order_status_display.value = order[2]
      order_payment_display.value = order[3]

      # Debug message to confirm order ID is set
      order_detail_status.value = f"Showing details for Order ID: {order[0]}"

      # Check if order has trade-in
      c.execute("""
        SELECT COUNT(*)
        FROM TRADE_IN
        WHERE ORDERS_OR_ID = :order_id
      """, {'order_id': selected_order_id.value})

      trade_in_count = c.fetchone()[0]

      if trade_in_count > 0:
        c.execute("""
          SELECT TR_STATUS
          FROM TRADE_IN
          WHERE ORDERS_OR_ID = :order_id
          AND ROWNUM = 1
        """, {'order_id': selected_order_id.value})

        trade_in_status = c.fetchone()[0]
        trade_in_status_display.value = trade_in_status
        view_trade_in_button.disabled = False
      else:
        trade_in_status_display.value = "No trade-in for this order"
        view_trade_in_button.disabled = True

      # Get order items
      c.execute("""
        SELECT p.P_NAME, ol.OL_P_QTY, p.P_PRICE, ol.OL_SUBTOTAL
        FROM ORDERLINE ol
        JOIN PRODUCT p ON ol.PRODUCT_P_ID = p.P_ID
        WHERE ol.ORDERS_OR_ID = :order_id
      """, {'order_id': selected_order_id.value})

      order_items = c.fetchall()

      if not order_items:
        with order_items_output:
          display(Label("No items found for this order."))
        return

      # Create styled header row
      header_row = widgets.HBox([
        Label("Product", layout={'width': '250px'}),
        Label("Quantity", layout={'width': '100px'}),
        Label("Price", layout={'width': '100px'}),
        Label("Subtotal", layout={'width': '100px'})
      ])

      # Style the header row
      for child in header_row.children:
        if isinstance(child, Label):
          child.style.font_weight = 'bold'

      item_rows = [header_row]

      total_price = 0

      for item in order_items:
        product_name = item[0]
        quantity = item[1]
        price = item[2]
        subtotal = item[3]

        total_price += subtotal

        # Create styled row for this item
        item_row = widgets.HBox([
          Label(product_name, layout={'width': '250px'}),
          Label(str(quantity), layout={'width': '100px'}),
          Label(f"${price:.2f}", layout={'width': '100px'}),
          Label(f"${subtotal:.2f}", layout={'width': '100px'})
        ], layout=widgets.Layout(margin='5px 0px'))

        item_rows.append(item_row)

      # Add styled total row
      total_row = widgets.HBox([
        Label("", layout={'width': '250px'}),
        Label("", layout={'width': '100px'}),
        Label("Total:", layout={'width': '100px', 'font_weight': 'bold'}),
        Label(f"${total_price:.2f}", layout={'width': '100px', 'font_weight': 'bold'})
      ], layout=widgets.Layout(margin='10px 0px 5px 0px'))

      item_rows.append(total_row)

      # Display all items in a styled container
      with order_items_output:
        display(widgets.VBox(item_rows,
                            layout=widgets.Layout(
                                border='1px solid #ddd',
                                padding='10px',
                                margin='10px 0px',
                                border_radius='5px')))

    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      order_detail_status.value = f"Database Error: {error_obj.message}"

  # Button click handlers
  def on_back_to_orders_button_clicked(b):
    showPage("Order List")

  def on_view_trade_in_button_clicked(b):
    showPage("Trade_In_Detail")

  back_to_orders_button.on_click(on_back_to_orders_button_clicked)
  view_trade_in_button.on_click(on_view_trade_in_button_clicked)

  # Styled layout
  order_detail_header = widgets.HBox([order_detail_title],
                                    layout=widgets.Layout(justify_content='center',
                                                         margin='10px 0px 20px 0px'))

  # Order info section with styling
  order_info = widgets.HBox([order_info_label],
                           layout=widgets.Layout(margin='15px 0px 10px 0px'))

  order_id_row = widgets.HBox([order_id_label, order_id_display],
                             layout=widgets.Layout(margin='5px 0px'))
  order_date_row = widgets.HBox([order_date_label, order_date_display],
                               layout=widgets.Layout(margin='5px 0px'))
  order_status_row = widgets.HBox([order_status_label, order_status_display],
                                 layout=widgets.Layout(margin='5px 0px'))
  order_payment_row = widgets.HBox([order_payment_label, order_payment_display],
                                  layout=widgets.Layout(margin='5px 0px'))

  # Order info container
  order_info_container = widgets.VBox([
    order_info,
    order_date_row,
    order_status_row,
    order_payment_row
  ], layout=widgets.Layout(
      border='1px solid #ddd',
      padding='10px',
      margin='10px 0px',
      border_radius='5px'))

  # Order items section with styling
  order_items_header = widgets.HBox([order_items_label],
                                   layout=widgets.Layout(margin='15px 0px 10px 0px'))

  # Trade-in section with styling
  trade_in_header = widgets.HBox([trade_in_label],
                                layout=widgets.Layout(margin='15px 0px 10px 0px'))

  trade_in_status_row = widgets.HBox([trade_in_status_label, trade_in_status_display],
                                    layout=widgets.Layout(margin='5px 0px'))

  trade_in_button_row = widgets.HBox([view_trade_in_button],
                                    layout=widgets.Layout(margin='10px 0px'))

  # Trade-in container
  trade_in_container = widgets.VBox([
    trade_in_header,
    trade_in_status_row,
    trade_in_button_row
  ], layout=widgets.Layout(
      border='1px solid #ddd',
      padding='10px',
      margin='10px 0px',
      border_radius='5px'))

  # Button row with styling
  buttons_row = widgets.HBox([back_to_orders_button],
                            layout=widgets.Layout(margin='15px 0px'))

  # Main layout
  display(widgets.VBox([
    order_detail_header,
    order_info_container,
    order_items_header,
    order_items_output,
    trade_in_container,
    order_detail_status,
    buttons_row
  ], layout=widgets.Layout(
      padding='15px',
      margin='10px')))

##############################################
############## Trade_In_Detail Tab ###########
##############################################

with tb_all.output_to('Trade_In_Detail'):
  # Create styled title
  trade_in_detail_title = Label("Trade-In Details")
  trade_in_detail_title.style.font_weight = 'bold'
  trade_in_detail_title.style.font_size = '18px'
  trade_in_detail_status = Label("")

  # Order information section with styling
  order_info_label = Label("Order Information")
  order_info_label.style.font_weight = 'bold'
  order_info_label.style.font_size = '16px'

  order_id_label = Label("Order ID: ", layout=widgets.Layout(width='150px'))
  order_id_display = widgets.HTML("", layout=widgets.Layout(width='250px'))

  # Trade-in items section with styling
  trade_in_items_label = Label("Trade-In Items")
  trade_in_items_label.style.font_weight = 'bold'
  trade_in_items_label.style.font_size = '16px'
  trade_in_items_output = widgets.Output()

  # Styled button
  back_to_order_detail_button = widgets.Button(description="Back to Order Detail",
                                              button_style='info',
                                              layout=widgets.Layout(width='180px'))

  # Function to show trade-in detail
  def show_trade_in_detail():
    trade_in_items_output.clear_output()

    try:
      # Display order ID
      order_id_display.value = selected_order_id.value

      # Get trade-in items
      c.execute("""
        SELECT TR_ID, TR_ITEM, TR_SERIAL_NUMBER, TR_ITEM_CONDITION, TR_ITEM_DECSRIPTION, TR_PRICE, TR_STATUS
        FROM TRADE_IN
        WHERE ORDERS_OR_ID = :order_id
      """, {'order_id': selected_order_id.value})

      trade_in_items = c.fetchall()

      if not trade_in_items:
        with trade_in_items_output:
          display(Label("No trade-in items found for this order."))
        return

      # Create styled header row
      header_row = widgets.HBox([
        Label("Item", layout={'width': '200px'}),
        Label("Serial Number", layout={'width': '150px'}),
        Label("Condition", layout={'width': '100px'}),
        Label("Description", layout={'width': '200px'}),
        Label("Price", layout={'width': '100px'}),
        Label("Status", layout={'width': '100px'})
      ])

      # Style the header row
      for child in header_row.children:
        if isinstance(child, Label):
          child.style.font_weight = 'bold'

      item_rows = [header_row]

      total_trade_in_value = 0

      for item in trade_in_items:
        tr_id = item[0]
        tr_item = item[1]
        tr_serial = item[2]
        tr_condition = item[3]
        tr_description = item[4]
        tr_price = item[5] if item[5] is not None else 0
        tr_status = item[6]

        total_trade_in_value += tr_price

        # Create styled row for this item
        item_row = widgets.HBox([
          Label(tr_item, layout={'width': '200px'}),
          Label(tr_serial, layout={'width': '150px'}),
          Label(tr_condition, layout={'width': '100px'}),
          Label(tr_description, layout={'width': '200px'}),
          Label(f"${tr_price:.2f}", layout={'width': '100px'}),
          Label(tr_status, layout={'width': '100px'})
        ], layout=widgets.Layout(margin='5px 0px'))

        item_rows.append(item_row)

      # Add styled total row
      total_row = widgets.HBox([
        Label("", layout={'width': '200px'}),
        Label("", layout={'width': '150px'}),
        Label("", layout={'width': '100px'}),
        Label("Total Trade-In Value:", layout={'width': '200px', 'font_weight': 'bold'}),
        Label(f"${total_trade_in_value:.2f}", layout={'width': '100px', 'font_weight': 'bold'}),
        Label("", layout={'width': '100px'})
      ], layout=widgets.Layout(margin='10px 0px 5px 0px'))

      item_rows.append(total_row)

      # Display all items in a styled container
      with trade_in_items_output:
        # Trade-in items table
        display(widgets.VBox(item_rows,
                            layout=widgets.Layout(
                                border='1px solid #ddd',
                                padding='10px',
                                margin='10px 0px',
                                border_radius='5px')))

      # Get order total before trade-in
      c.execute("""
        SELECT NVL(SUM(OL_TOTALPRICE), 0)
        FROM ORDERLINE
        WHERE ORDERS_OR_ID = :order_id
      """, {'order_id': selected_order_id.value})

      order_total = c.fetchone()[0]

      # Display final price after trade-in in a styled summary box
      final_price = max(0, order_total - total_trade_in_value)

      with trade_in_items_output:
        # Create styled summary container
        summary_container = widgets.VBox([
          Label(f"Order Total: ${order_total:.2f}"),
          Label(f"Trade-In Discount: ${min(total_trade_in_value, order_total):.2f}"),
          Label(f"Final Price: ${final_price:.2f}")
        ], layout=widgets.Layout(
            border='1px solid #ddd',
            padding='15px',
            margin='15px 0px',
            border_radius='5px',
            background_color='#f8f9fa'))

        # Style the summary labels
        for child in summary_container.children:
          if isinstance(child, Label):
            child.style.font_weight = 'bold'
            if "Final Price" in child.value:
              child.style.color = '#28a745'  # Green color for final price

        display(summary_container)

    except cx_Oracle.DatabaseError as e:
      error_obj, = e.args
      trade_in_detail_status.value = f"Database Error: {error_obj.message}"

  # Button click handlers
  def on_back_to_order_detail_button_clicked(b):
    showPage("Order Detail")

  back_to_order_detail_button.on_click(on_back_to_order_detail_button_clicked)

  # Styled layout
  trade_in_detail_header = widgets.HBox([trade_in_detail_title],layout=widgets.Layout(justify_content='center',margin='10px 0px 20px 0px'))

  # Order info section with styling
  order_info = widgets.HBox([order_info_label],layout=widgets.Layout(margin='15px 0px 10px 0px'))

  order_id_row = widgets.HBox([order_id_label, order_id_display],layout=widgets.Layout(margin='5px 0px'))

  # Order info container
  order_info_container = widgets.VBox([
    order_info,
    order_id_row
  ], layout=widgets.Layout(
      border='1px solid #ddd',
      padding='10px',
      margin='10px 0px',
      border_radius='5px'))

  # Trade-in items section with styling
  trade_in_items_header = widgets.HBox([trade_in_items_label],layout=widgets.Layout(margin='15px 0px 10px 0px'))

  # Button row with styling
  buttons_row = widgets.HBox([back_to_order_detail_button],layout=widgets.Layout(margin='15px 0px'))

  # Main layout
  display(widgets.VBox([
    trade_in_detail_header,
    order_info_container,
    trade_in_items_header,
    trade_in_items_output,
    trade_in_detail_status,
    buttons_row
  ], layout=widgets.Layout(
      padding='15px',
      margin='10px')))

# Initialize the application by showing the Introduction page
showPage("Introduction")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>